## Parameters

In [1]:
import math
from sage.rings.polynomial.polynomial_zmod_flint import Polynomial_zmod_flint as polynomial
from sage.matrix.constructor import Matrix as matrix

# Kyber Parameters
q = 3329
k = 2
n = 256

RQ.<x> = GF(q)[]
RQ.gen()
f = x^(n+1) + 1

DEBUG = True

In [2]:
# Helper Functions
def reducePolynomials(matrx):
    cols = len(matrx.columns())
    rows = len(matrx.rows())
    out = [[matrx.coefficient((i,j)) for j in range(cols)] for i in range(rows)]
    for row in range(rows):
        for col in range(cols):
            # Divide the polynomial in out[row][col] by f, only keep the remainder
            _, rt = out[row][col].quo_rem(f)
            out[row][col] = rt
    return matrix(RQ, out)

def randomList(length, cbd=False):
    if cbd:
        # NOT real centered binomial distrbution!
        return [randrange(2) - randrange(2) for i in range(length)]
    else:
        return [randrange(q) for i in range(length)]

def randomPolyUniform(length):
    return RQ(randomList(length))

def randomPolyCbd(length):
    return RQ(randomList(length, cbd=True))

def compress(poly):
     q2 = math.ceil(q/2)
     return poly * q2

def decompress(poly) -> int:
    return [(1 if 3*(q/4) > Integer(i) > q/4 else 0) for i in poly]
    # return int(''.join([str(x) for x in dpoly]), 2)

def dbg(s: str = ''):
    if DEBUG:
        print(s)

## K-PKE KeyGen (INDCPA)

In [3]:
def kpke_keygen() -> (matrix, matrix, matrix):
    dbg('===== kpke_keygen =====')
    # A is a k*k dimension matrix of polynomials with n terms
    A = []
    for _ in range(0, k):
        tA = []
        for _ in range(0, k):
            tA.append(randomPolyUniform(n))
        A.append(tA)
    A = matrix(A)
    dbg('A:')
    dbg(A)
    
    # s is a k*1 dimension matrix of polynomials with n terms
    s = [[randomPolyCbd(n)] for _ in range(0, k)]
    s = matrix(s)
    dbg('s:')
    dbg(s)

    # e is a k*1 dimension matrix of polynomials with n terms
    e = [[randomPolyCbd(n)] for _ in range(0, k)]
    e = matrix(e)
    dbg('e:')
    dbg(e)

#   A*s is a k * 1 matrix of polynomials with n terms
#   A*s+e is a k * 1 matrix polynomials with n terms

#   Example when k=2:
#   |     A     |   |  s  |   |  e  |
#   | :-- | :-- |   | :-- |   | :-- |
#   | 0,0 | 0,1 |   |  0  |   |  0  |
#   | 1,0 | 1,1 |   |  1  |   |  1  |

#   |             A * s             |
#   | :---------------------------- |
#   | A[0,0] * s[0] + A[0,1] * s[1] |
#   | A[1,0] * s[0] + A[1,1] * s[1] |

#   |     As+e     |
#   | :----------- |
#   | As[0] + e[0] |
#   | As[1] + e[1] |

    # compute t = A*s*e
    # t is a k*1 dimension matrix
    # Reduce
    t = reducePolynomials(A*s+e)
    dbg()

    return (A, t, s)

## K-PKE Encrypt (INDCPA)

In [4]:
def kpke_encrypt(A: matrix, t: matrix, m: int) -> (polynomial, polynomial):
    dbg('===== kpke_encrypt =====')
    # Ensure that m does not have more bits than n bits
    if len(m.bits()) > n:
        raise ValueError('m has more bits than n!')
    mm = m.bits()
    dbg('Bits of m:')
    dbg(mm)
    
    # We need m to be at least n bits long.
    # Pad mm with 0s until desired length is reached
    pad = [0 for _ in range(0, n - len(mm))]
    mm = RQ(mm + pad)
    dbg('Polynomial m:')
    dbg(mm)
    mm = compress(mm)
    dbg('Compressed m:')
    dbg(mm)

    # Generate r, e1, e2
    # r is a k*1 matrix
    r = [[randomPolyCbd(n)] for _ in range(0, k)]
    r = matrix(r)
    dbg('r:')
    dbg(r)

    # e1 is a k*1 matrix
    e1 = [[randomPolyCbd(n)] for _ in range(0, k)]
    e1 = matrix(e1)
    dbg('e1:')
    dbg(e1)

    # e2 is an n-length polynomial
    e2 = randomPolyCbd(n)
    dbg('e2:')
    dbg(e2)

    u = A.transpose() * r + e1
    v = t.transpose() * r + e2 + mm

    u = reducePolynomials(u)
    v = reducePolynomials(v)

    dbg('u:')
    dbg(u)
    dbg('v:')
    dbg(v)
    dbg()

    return (u, v)

    

## K-PKE Decryption

In [5]:
def kpke_decrypt(u: matrix, v: matrix, s: matrix) -> int:
    dbg('===== kpke_decrypt =====')
    # Compute a noisy result mn
    mn = v - s.transpose() * u
    mn = (reducePolynomials(mn)).coefficients()[0]
    dbg('Noisy recovered m:')
    dbg(mn)

    mn_c = mn.coefficients(sparse=false)
    mn_c.reverse()

    m_rec = decompress(mn_c)
    dbg('Decompressed m:')
    dbg(m_rec)

    m_rec = int(''.join([str(x) for x in m_rec]), 2)
    dbg('Recovered m:')
    dbg(m_rec)
    dbg()

    return m_rec

In [6]:
from os import urandom
m = Integer(int.from_bytes(urandom(((n+7) & (-8))//8), 'big'))
m &= 2**n-1


A, t, s = kpke_keygen()
u, v = kpke_encrypt(A, t, m)
mr = kpke_decrypt(u, v, s)
dbg('Original m:')
dbg(m)
dbg(m.bits())
if m != mr:
    raise ValueError('decrypted m does not match, final decompression likely failed')

===== kpke_keygen =====
A:
[                          3327*x^255 + 529*x^254 + 744*x^253 + 2036*x^252 + 3200*x^251 + 1640*x^250 + 3327*x^249 + 2243*x^248 + 1478*x^247 + 589*x^246 + 1381*x^245 + 1283*x^244 + 2325*x^243 + 3063*x^242 + 2281*x^241 + 899*x^240 + 1863*x^239 + 1166*x^238 + 3041*x^237 + 2071*x^236 + 2670*x^235 + 176*x^234 + 1724*x^233 + 387*x^232 + 2779*x^231 + 137*x^230 + 1359*x^229 + 2092*x^228 + 2342*x^227 + 283*x^226 + 1281*x^225 + 486*x^224 + 997*x^223 + 325*x^222 + 2716*x^221 + 1220*x^220 + 845*x^219 + 1101*x^218 + 1769*x^217 + 2174*x^216 + 1543*x^215 + 770*x^214 + 1978*x^213 + 2786*x^212 + 771*x^211 + 2458*x^210 + 1844*x^209 + 546*x^208 + 981*x^207 + 2500*x^206 + 1169*x^205 + 879*x^204 + 289*x^203 + 1023*x^202 + 84*x^201 + 168*x^200 + 1333*x^199 + 2494*x^198 + 3228*x^197 + 458*x^196 + 2273*x^195 + 1279*x^194 + 785*x^193 + 1722*x^192 + 395*x^191 + 3233*x^190 + 1384*x^189 + 1977*x^188 + 1611*x^187 + 486*x^186 + 1362*x^185 + 822*x^184 + 2977*x^183 + 1293*x^182 + 950*x^181 